In [1]:
import os
import ast
import re
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor


def extract_definitions(file_path, file_extension):
    """Extract defined variables and functions from Python, Svelte, and TypeScript files."""
    definitions = {}

    # Python file handling using AST
    if file_extension == '.py':
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                definitions[node.name] = {'type': 'function', 'context': code}
            elif isinstance(node, ast.Assign):
                for target in node.targets:
                    if isinstance(target, ast.Name) and isinstance(target.ctx, ast.Store):
                        definitions[target.id] = {'type': 'variable', 'context': code}

    # Svelte and TypeScript file handling using Regex (basic detection)
    elif file_extension in ['.svelte', '.ts']:
        with open(file_path, 'r', encoding='utf-8') as f:
            code = f.read()

        # Regex for functions (e.g., function foo() {...} or const foo = () => {...})
        func_pattern = r'\b(?:function\s+(\w+)|const\s+(\w+)\s*=\s*\()'
        var_pattern = r'\b(?:let|const|var)\s+(\w+)'

        functions = re.findall(func_pattern, code)
        variables = re.findall(var_pattern, code)

        for func in functions:
            name = func[0] if func[0] else func[1]
            definitions[name] = {'type': 'function', 'context': code}

        for var in variables:
            definitions[var] = {'type': 'variable', 'context': code}

    return definitions


def suggest_better_name(name, def_type, context):
    """Simulate LLM suggesting a snake_case name based on context."""
    if def_type == 'function':
        return f"process_{name.lower()}" if not name.islower() else name
    elif def_type == 'variable':
        return f"{name.lower()}_value" if not name.islower() else name
    return name  # Fallback


def generate_rename_map(directory):
    """Generate a rename map from Python, Svelte, and TypeScript files in a directory."""
    all_definitions = {}
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_extension = os.path.splitext(file)[1].lower()
            if file_extension in ['.py', '.ts', '.svelte']:
                defs = extract_definitions(file_path, file_extension)
                all_definitions[file_path] = defs

    rename_map = {}
    for file_path, defs in all_definitions.items():
        for name, info in defs.items():
            if name not in rename_map:  # Only suggest once per unique name
                new_name = suggest_better_name(name, info['type'], info['context'])
                if new_name != name and not new_name.startswith('_'):  # Avoid private names
                    rename_map[name] = new_name

    return rename_map


def replace_in_file(file_path, rename_map):
    """Replace occurrences in a file using regex with word boundaries."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        original_content = content
        # Create a regex pattern that matches any of the keys with word boundaries
        pattern = r'\b(?:' + '|'.join(re.escape(key) for key in rename_map.keys()) + r')\b'

        def replacer(match):
            return rename_map[match.group(0)]

        new_content = re.sub(pattern, replacer, content)

        # Only write if content changed
        if new_content != original_content:
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)
            return True
        return False
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return False


def process_directory(directory, rename_map, batch_size=10):
    """Process all Python, Svelte, and TypeScript files in the directory in batches."""
    # Define a list of directories to exclude (case insensitive)
    exclude_dirs = ['code_helper']

    py_ts_svelte_files = []
    for root, _, files in os.walk(directory):
        # Skip directories that contain any name in the exclude_dirs list
        if any(excluded_dir.lower() in root.lower() for excluded_dir in exclude_dirs):
            continue  # Skip this directory and its contents

        for file in files:
            if file.endswith(('.py', '.ts', '.svelte')):
                py_ts_svelte_files.append(os.path.join(root, file))

    if not py_ts_svelte_files:
        print("No Python, Svelte, or TypeScript files found in the directory.")
        return

    results = [False] * len(py_ts_svelte_files)

    # Function to be executed by each thread
    def process_file(index, file_path):
        results[index] = replace_in_file(file_path, rename_map)

    # Process files in batches
    total_files = len(py_ts_svelte_files)
    with tqdm(total=total_files, desc="Replacing variables/functions", unit="file") as pbar:
        for i in range(0, total_files, batch_size):
            batch_files = py_ts_svelte_files[i:i + batch_size]
            with ThreadPoolExecutor(max_workers=os.cpu_count() or 4) as executor:
                futures = []
                for j, file_path in enumerate(batch_files):
                    future = executor.submit(process_file, j, file_path)
                    future.add_done_callback(lambda p: pbar.update(1))
                    futures.append(future)

                # Wait for all tasks to complete
                for future in futures:
                    try:
                        future.result()
                    except Exception as e:
                        print(f"Thread error: {e}")

    print(f"Completed renaming in {total_files} files.")


if __name__ == "__main__":
    project_dir = input("Enter your project directory: ")
    rename_map = generate_rename_map(project_dir)

    # Save the rename map to a JSON file for review
    with open("rename_map.json", 'w', encoding='utf-8') as f:
        json.dump(rename_map, f, indent=4)

    # Ask for batch size
    batch_size = int(input("Enter batch size for applying changes: "))

    # Process directory in batches
    process_directory(project_dir, rename_map, batch_size)

Replacing variables/functions: 100%|██████████| 518/518 [00:07<00:00, 73.01file/s] 

Completed renaming in 518 files.
